# Transfer learning

In [1]:
APPLY = True
EAGER = False
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'sample_transfer_learning'
MODALITY = 'text'

In [2]:
from superduper import superduper, CFG

db = superduper('mongomock://test_db')

2025-Mar-22 15:40:00.38| INFO     | Duncans-MacBook-Pro.local| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Mar-22 15:40:00.49| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:51   | Building Data Layer
2025-Mar-22 15:40:00.49| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:68   | Data Layer built
2025-Mar-22 15:40:00.49| INFO     | Duncans-MacBook-Pro.local| superduper.backends.base.cluster:109  | Cluster initialized in 0.00 seconds.
2025-Mar-22 15:40:00.49| INFO     | Duncans-MacBook-Pro.local| superduper.base.build:148  | Configuration: 
 +----------------+-------------------------------+
| Configuration  |             Value             |
+----------------+-------------------------------+
|  Data Backend  |      mongomock://test_db      |
| Artifact Store | filesystem://./artifact_store |
|     Cache      |           in-process          |
+----------------+-------------------------------+


<!-- TABS -->
## Get useful sample data

In [3]:
def getter():
    import json
    import random
    import subprocess

    subprocess.run([
        'curl', '-O', 'https://superduperdb-public-demo.s3.amazonaws.com/text_classification.json',
    ])
    with open("text_classification.json", "r") as f:
        data = json.load(f)
    subprocess.run(['rm', 'text_classification.json'])
    data = data[:200]
    def fold(): return {True: 'valid', False: 'train'}[random.random() < 0.1]
    data = [{**r, '_fold': fold()} for r in data]
    return data

After obtaining the data, we insert it into the database.

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [4]:
if APPLY:
    data = getter()
    from superduper import Table

    db.apply(
        Table(
            COLLECTION_NAME,
            fields={'x': 'str', 'y': 'int'},
        ),
        force=True,
    )
    
    ids = db[COLLECTION_NAME].insert(data)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  1 1298k    1 17003    0     0  16151      0  0:01:22  0:00:01  0:01:21 16208

2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'sample_transfer_learning')) from metadata...
2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:94   | Found these changes and/ or additions that need to be made:
2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:96   | ----------------------------------------------------------------------------------------------------
2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:97   | METADATA EVENTS:
2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:98   | ----------------------------------------------------------------------------------------------------
2025-Mar-22 15:40:15.47| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:109  | [0]: Table:sample_transfer_learning:ef45ff296b338bb99857f0fc7d0d3416: create
2025-Mar-22 15:40:15.47| I

100 1298k  100 1298k    0     0   711k      0  0:00:01  0:00:01 --:--:--  713k


<!-- TABS -->
## Compute features

In [5]:
import sentence_transformers
from superduper import Listener
from superduper_sentence_transformers import SentenceTransformer


embedding = SentenceTransformer(
    identifier="embedding",
    model='all-MiniLM-L6-v2',
    postprocess=lambda x: x.tolist(),
)

[2025-03-22 15:40:21] datasets INFO PyTorch version 2.5.1 available.
[2025-03-22 15:40:21] sentence_transformers.SentenceTransformer INFO Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [7]:
feature_extractor_listener = Listener(
    model=embedding,
    select=db[COLLECTION_NAME],
    key='x',
    identifier="features"
)


if APPLY and EAGER:
    feature_extractor_listener = db.apply(
        feature_extractor_listener,
        force=True,
    )

<!-- TABS -->
## Build and train classifier

In [8]:
from superduper_sklearn import Estimator, SklearnTrainer
from sklearn.svm import SVC


scikit_model = Estimator(
    identifier="my-model-scikit",
    object=SVC(),
    trainer=SklearnTrainer(
        "my-scikit-trainer",
        key=(feature_extractor_listener.outputs, "y"),
        select=db[COLLECTION_NAME].outputs(feature_extractor_listener.predict_id),
    ),
    upstream=[feature_extractor_listener],
)

Define a validation for evaluating the effect after training.

In [9]:
from superduper import Dataset, Metric, Validation

def acc(x, y):
    return sum([xx == yy for xx, yy in zip(x, y)]) / len(x)

accuracy = Metric(identifier="acc", object=acc)

t = db[COLLECTION_NAME]
select = t.filter(t['_fold'] == 'valid').outputs(feature_extractor_listener.predict_id)

validation = Validation(
    "transfer_learning_performance",
    key=(feature_extractor_listener.outputs, "y"),
    datasets=[
        Dataset(
            identifier="my-valid",
            select=select,
        )
    ],
    metrics=[accuracy],
)
scikit_model.validation = validation

If we execute the apply function, then the model will be added to the database, and because the model has a Trainer, it will perform training.

In [10]:
if APPLY and EAGER:
    db.apply(scikit_model, force=True)

Get the training metrics

In [12]:
from superduper import Application

application = Application(
    identifier='transfer-learning',
    components=[feature_extractor_listener, scikit_model],
)

In [13]:
if APPLY:
    db.apply(application)

2025-Mar-22 15:40:57.31| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Application', 'transfer-learning')) from metadata...
2025-Mar-22 15:40:57.31| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Listener', 'features')) from metadata...
2025-Mar-22 15:40:57.31| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('SentenceTransformer', 'embedding')) from metadata...
2025-Mar-22 15:40:58.88| WARNING  | Duncans-MacBook-Pro.local| superduper.base.artifacts:243  | File ./artifact_store/141a2f7ec61bf77731f5b2c8eb3c841e55b682660032932ce2eee4c54b7afd33 already exists
2025-Mar-22 15:40:58.88| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', '_outputs__features__c1b5d86988903f267e496024ca7b7ad2')) from metadata...
2025-Mar-22 15:40:58.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Estimator', 'my-model-scikit')) from metadata...
2025

2025-Mar-22 15:41:00.15| INFO     | Duncans-MacBook-Pro.local| superduper.base.event:142  | Creating superduper_sentence_transformers.model.SentenceTransformer:embedding:34fa3ac0f2f9c2363b1ccab9b2b3f230
2025-Mar-22 15:41:00.15| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'ArtifactRelations')) from metadata...
2025-Mar-22 15:41:00.16| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'Table')) from metadata...
2025-Mar-22 15:41:00.16| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'SentenceTransformer')) from metadata...
2025-Mar-22 15:41:00.17| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'ParentChildAssociations')) from metadata...
2025-Mar-22 15:41:00.17| INFO     | Duncans-MacBook-Pro.local| superduper.base.event:142  | Creating superduper.components.table.Table:_outputs__features__c1b5d86988903f267e496024ca7b7ad2:3fca116fb

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

2025-Mar-22 15:41:01.38| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', '_outputs__features__c1b5d86988903f267e496024ca7b7ad2')) from metadata...
2025-Mar-22 15:41:01.41| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'Job')) from metadata...
2025-Mar-22 15:41:01.45| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'Job')) from metadata...
2025-Mar-22 15:41:01.61| INFO     | Duncans-MacBook-Pro.local| superduper_sklearn.model:42   | Loading dataset into memory for Estimator.fit


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 3405467.01it/s]


2025-Mar-22 15:41:03.23| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Estimator', 'my-model-scikit')) from metadata...
2025-Mar-22 15:41:03.24| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Listener', 'features')) from metadata...
2025-Mar-22 15:41:03.24| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('SentenceTransformer', 'embedding')) from metadata...
2025-Mar-22 15:41:04.87| WARNING  | Duncans-MacBook-Pro.local| superduper.base.artifacts:243  | File ./artifact_store/141a2f7ec61bf77731f5b2c8eb3c841e55b682660032932ce2eee4c54b7afd33 already exists
2025-Mar-22 15:41:04.87| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', '_outputs__features__c1b5d86988903f267e496024ca7b7ad2')) from metadata...
2025-Mar-22 15:41:04.88| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Validation', 'transfer_learning_performance')) from metad

my-model-scikit
├── my-model-scikit
│   └── status: update
├── features
│   └── status: update
└── embedding
    └── status: update

2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:94   | Found these changes and/ or additions that need to be made:
2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:96   | ----------------------------------------------------------------------------------------------------
2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:97   | METADATA EVENTS:
2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:98   | ----------------------------------------------------------------------------------------------------
2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:105  | [0]: SentenceTransformerembedding:34fa3ac0f2f9c2363b1ccab9b2b3f230: update ~ [1]
2025-Mar-22 15:41:04.89| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:105  | [1]: Listenerfeatures:c1b5d86988903f267e496024ca7b7ad2: update ~ [2]
2025-Mar-22 15:41:0

In [14]:
if APPLY:
    model = db.load('Estimator', 'my-model-scikit')
    print(model.metric_values)

2025-Mar-22 15:41:09.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Estimator', 'my-model-scikit')) from metadata...
{'my-valid/acc': 0.8695652173913043}


In [16]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

t = Template(
    'transfer_learning',
    default_tables=[Table(
        'sample_transfer_learning',
        fields={'x': 'str', 'y': 'int'},
        data=RemoteData(
            'text_classification',
            getter=getter,
        ),
    )],
    template=application,
    substitutions={'docs': 'table_name', 'text': 'modality'},
    template_variables=['table_name', 'framework', 'modality'],
    types={
        'table_name': {
            'type': 'str',
            'default': 'sample_transfer_learning',
        },
        'modality': {
            'type': 'str',
            'default': 'text',
        },
        'framework': {
            'type': 'str',
            'default': 'scikit-framework',
        },
    },
    db=db
)

In [17]:
t.export('.')